Extrae el % de bateria del comentario y lo pega en la columna de bateria

In [1]:
import pandas as pd
import re
from datetime import datetime

In [2]:
elemento_a_buscar = "iphone"

carpeta = "4_download_description"
fecha = datetime.now().strftime("%Y%m%d")
fecha = "20250117"

In [3]:

nombre_archivo = fecha + '_' + elemento_a_buscar
df_original = pd.read_csv(carpeta + '/' + nombre_archivo + '.csv')
df_original.drop('Unnamed: 0', axis=1, inplace=True)
df_original.head(2)

,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,scrap_locat,dist_scrap_locat,reservado,url,tiene_color,tiene_emojis,tiene_revisado,comentario
0,1033637533,20250117,14,basic,128,NaN,"615,37",good,apple iphone 14 128gb amarillo - revisado,madrid,60,False,https://es.wallapop.com/item/apple-iphone-14-1...,True,True,True,NaN
1,1041328668,20250117,14,pro,256,NaN,"755,00",as_good_as_new,iphone 14 pro 256gb,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-256...,False,False,False,Detalles del producto\nVendo Iphone 14 pro 256...


In [4]:
df = df_original.copy()

In [5]:
def encontrar_bateria(texto: str):
    # Verificar si el texto es NaN o similar
    if pd.isna(texto):
        return None
    # Buscar números seguidos por %
    numeros_con_porcentaje = re.findall(r'\b(8[0-9]|9[0-9]|100)\b%', texto)
    # Si encuentra números con %, devolver el primero
    if numeros_con_porcentaje:
        return numeros_con_porcentaje[0] 
    # Si no hay % buscar solo los números
    numeros = re.findall(r'\b(8[0-9]|9[0-9]|100)\b', texto)
    # Si encuentra números, devolver el primero
    if numeros:
        return numeros[0]
    # Si no encuentra ningún número, devolver None
    return None

In [6]:
# buscar 128|256|512 seguido de \s*(Mb|MB|mb)
# si no, buscar \b1[Tt]\b
# si no, buscar \b(128|256|512)\b
def encontrar_memoria(texto: str) -> str:
    if pd.isna(texto):
        return None
    texto = str(texto)
    
    # Buscar 128|256|512 seguido de \s*(Mb|MB|mb)
    memorias_con_unidades = re.search(r'\b(128|256|512)\b\s*(Mb|MB|mb)', texto)
    if memorias_con_unidades:
        memoria, unidad = memorias_con_unidades.groups()
        return memoria + unidad
    
    # Buscar \b1[Tt]\b
    memoria_1t = re.search(r'\b1[Tt]', texto)
    if memoria_1t:
        return '_1T'
    
    # Buscar \b(128|256|512)\b
    memorias_sin_unidades = re.search(r'\b(128|256|512)\b', texto)
    if memorias_sin_unidades:
        return memorias_sin_unidades.group(0)
    
    return None
    
def es_tienda(texto: str) -> bool:
    if pd.isna(texto):
        return None
    words_to_find = ('reacondicionado', 'tienda')
    for word in words_to_find:
        if word in texto:
            return True 
    return False 

def tiene_garantia(texto: str):
    if pd.isna(texto):
        return None
    words_to_find = ('garantia', 'care', 'factura')
    for word in words_to_find:
        if word in texto:
            return True 
    return False 

In [7]:

missing_values = df_original.isnull().sum()
print(missing_values)
df['bateria'] = df_original['comentario'].apply(encontrar_bateria)
df['memoria'] = df_original['comentario'].apply(encontrar_memoria)
df['tienda'] = df_original['comentario'].apply(es_tienda)
df['garantia'] = df_original['comentario'].apply(tiene_garantia)

missing_values = df.isnull().sum()
print(missing_values)


id                     0
fecha                  0
gen                    0
mod                    0
memoria              416
bateria             1163
precio                 0
estado                 0
nombre                 0
scrap_locat            0
dist_scrap_locat       0
reservado              0
url                    0
tiene_color            0
tiene_emojis           0
tiene_revisado         0
comentario           198
dtype: int64
id                    0
fecha                 0
gen                   0
mod                   0
memoria             517
bateria             712
precio                0
estado                0
nombre                0
scrap_locat           0
dist_scrap_locat      0
reservado             0
url                   0
tiene_color           0
tiene_emojis          0
tiene_revisado        0
comentario          198
tienda              198
garantia            198
dtype: int64


In [8]:
df['tienda'].describe()

count      1068
unique        2
top       False
freq        882
Name: tienda, dtype: object

In [9]:
carpeta = "5_data_from_comments"
df.to_csv(carpeta + '/' + nombre_archivo + '.csv')